<a href="https://colab.research.google.com/github/Courage-gif/Deep-Learning_-NLP/blob/main/Document_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Courage Laizan

R227552Q

Link To the summary  document - https://docs.google.com/document/d/15XS-3q4zGqqyM9SnpCkR9LCLJkiuzZAg/edit?usp=sharing&ouid=116110048755074416183&rtpof=true&sd=true

In [3]:
# INSTALL LIBRARIES (FAST)
# ============================================
!pip install -q pdfminer.six bertopic transformers sentence-transformers \
               scikit-learn gensim umap-learn hdbscan python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 143.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 26.6 MB/s eta 0:00:00


In [ ]:
# ============================================
# GLOBAL DOCUMENT SUMMARY
# ============================================
import os
import warnings
from pdfminer.high_level import extract_text
from transformers import pipeline
import torch
from google.colab import drive
from docx import Document

warnings.filterwarnings("ignore")

# ============================================
# INITIAL SETUP
# ============================================
# Mount Google Drive
drive.mount("/content/drive")

# Configuration
PDF_FILE_PATH = "/content/drive/MyDrive/merged_documents.pdf"

# GPU Configuration
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using: {torch.cuda.get_device_name() if torch.cuda.is_available() else 'CPU'}")

# ============================================
# DATA EXTRACTION
# ============================================
print("📖 Extracting document content...")
full_text = extract_text(PDF_FILE_PATH)
print(f"📄 Document size: {len(full_text)} characters")

# ============================================
# SMART SUMMARIZATION
# ============================================
print("🧠 Generating intelligent summary...")

# Use a powerful summarization model
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",  # Excellent for document summarization
    device=device
)

# Smart chunking for better context
def create_smart_chunks(text, max_chunk_size=1024, overlap=100):
    """Split text into chunks with sentence boundaries"""
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk + sentence) < max_chunk_size:
            current_chunk += sentence + ". "
        else:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

# Create chunks and summarize
chunks = create_smart_chunks(full_text)
print(f"📋 Created {len(chunks)} content chunks")

# Generate summaries for each chunk
summaries = []
print("⏳ Processing content...")

for i, chunk in enumerate(chunks[:20]):  # Limit for reasonable processing time
    try:
        if len(chunk.split()) > 50:  # Only summarize substantial chunks
            summary = summarizer(
                chunk,
                max_length=150,
                min_length=60,
                do_sample=False,
                truncation=True
            )[0]["summary_text"]
            summaries.append(summary)
            print(f"✅ Processed chunk {i+1}/{min(len(chunks), 20)}")
    except Exception as e:
        print(f"⚠️ Skipped chunk {i+1}: {str(e)}")
        continue

# Combine summaries into coherent global summary
if summaries:
    # Create a final summary of summaries for coherence
    combined_text = " ".join(summaries)

    if len(combined_text.split()) > 200:
        try:
            global_summary = summarizer(
                combined_text,
                max_length=300,
                min_length=150,
                do_sample=False
            )[0]["summary_text"]
        except:
            global_summary = combined_text
    else:
        global_summary = combined_text

    print("🎉 Summary generation completed!")
else:
    global_summary = "Unable to generate summary from the provided document."
    print("❌ No summaries generated")

# ============================================
# DOCUMENT INSIGHTS
# ============================================
print("📊 Generating document insights...")

# Calculate document statistics
total_chars = len(full_text)
total_words = len(full_text.split())
total_sentences = full_text.count('.') + full_text.count('!') + full_text.count('?')
avg_sentence_length = total_words / max(total_sentences, 1)

# Estimate reading time (average reading speed: 200-250 words per minute)
reading_time_minutes = total_words / 200

# Document complexity assessment
def assess_complexity(text):
    words = text.split()
    if len(words) < 1000:
        return "Brief Document"
    elif len(words) < 5000:
        return "Medium Length"
    elif len(words) < 15000:
        return "Comprehensive Document"
    else:
        return "Extensive Research Document"

document_complexity = assess_complexity(full_text)

# ============================================
# CREATE ENGAGING REPORT
# ============================================
print("📝 Creating engaging summary report...")

doc = Document()
doc.add_heading('📚 Document Intelligence Report', 0)

# Executive Overview
doc.add_heading('🎯 Executive Overview', level=1)
doc.add_paragraph("This report provides an intelligent summary and key insights extracted from your document using advanced AI analysis.")

# Key Statistics
doc.add_heading('📈 Document Statistics', level=1)
stats_table = doc.add_table(rows=5, cols=2)
stats_table.style = 'Light Shading Accent 1'

stats_data = [
    ["Document Size", f"{total_words:,} words"],
    ["Estimated Reading Time", f"{reading_time_minutes:.1f} minutes"],
    ["Document Complexity", document_complexity],
    ["Sentences Analyzed", f"{total_sentences:,}"],
    ["Average Sentence Length", f"{avg_sentence_length:.1f} words"]
]

for i, (metric, value) in enumerate(stats_data):
    stats_table.cell(i, 0).text = metric
    stats_table.cell(i, 1).text = value

# Global Summary
doc.add_heading('🧠 Summary', level=1)
summary_paragraph = doc.add_paragraph()
summary_paragraph.add_run("Here's what this document is about:\n\n").bold = True
summary_paragraph.add_run(global_summary)

# Key Highlights Section
doc.add_heading('💡 Key Highlights', level=1)

highlight_paragraph = doc.add_paragraph()
highlight_paragraph.add_run("Based on the analysis:\n\n").bold = True

# Add dynamic highlights based on content
highlights = [
    f"• This is a {document_complexity.lower()} with substantial content",
    f"• The document contains approximately {total_sentences} key points",
    f"• Average complexity level based on sentence structure",
    f"• Suitable for readers interested in comprehensive analysis" if total_words > 3000 else "• Concise document ideal for quick reading"
]

for highlight in highlights:
    doc.add_paragraph(highlight)

# Reading Recommendations
doc.add_heading('📖 Reading Guide', level=1)
guide_paragraph = doc.add_paragraph()
guide_paragraph.add_run("For optimal understanding:\n\n").bold = True

if reading_time_minutes > 10:
    guide_text = f"• Set aside {reading_time_minutes:.0f} minutes for thorough reading\n• Focus on key sections highlighted in the summary\n• Consider breaking into multiple sessions for better retention"
else:
    guide_text = f"• Quick read: Approximately {reading_time_minutes:.0f} minutes\n• Ideal for immediate comprehension\n• Suitable for briefing and discussion"

doc.add_paragraph(guide_text)

# Save to Google Drive
drive_folder = "/content/drive/MyDrive/DeepLearning_Projects"
os.makedirs(drive_folder, exist_ok=True)

OUTPUT_PATH = os.path.join(drive_folder, "Document_Summary.docx")
doc.save(OUTPUT_PATH)

# ============================================
# FINAL OUTPUT
# ============================================
print(f"\n🎉 SUCCESS! Global summary report generated!")
print(f"📁 Saved to: {OUTPUT_PATH}")
print(f"📊 Document analyzed: {total_words:,} words")
print(f"⏱️  Reading time: {reading_time_minutes:.1f} minutes")
print(f"📑 Summary length: {len(global_summary.split())} words")
print(f"📋 Complexity: {document_complexity}")

print("\n" + "="*60)
print("🧠 YOUR DOCUMENT IN A NUTSHELL:")
print("="*60)
print(global_summary)
print("="*60)